In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-11-24"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
17489,2024-11-24,Turquia Tbl,11:15,Gaziantep,Kapakli Spor,1.44,2.52,171.5,1.81,1.85,-6.5,2.01,1.68,KG3yy22E,0.694444,0.396825,0.552486,0.540541,0.091270,129.166,49.798710,0.385540,0.6,1.341641,2.236068,124.80,1.636,0.482006,0.294625,-14.0,230.098,125.739657,0.546461,1.8,1.643168,0.912871,254.81,2.892,1.835407,0.634650,-16.0,80,83,1.56,3.07,114.754,316.264,0.385695,0.015456,0.126474,-3.99,-0.798,-0.551378,0.682417,0.6,-0.082417,-3.77,-0.754,-2.015915,0.349674,0.2,-0.149674
17490,2024-11-24,Turquia Tbl,09:00,Esenler Erokspor,Konya BBSK,1.36,2.81,160.5,1.83,1.83,-7.5,1.97,1.71,IZB6D74j,0.735294,0.355872,0.546448,0.546448,0.091166,113.668,9.996815,0.087947,2.4,1.341641,0.559017,117.66,1.378,0.167690,0.121691,47.0,150.738,41.528523,0.275501,1.2,1.643168,1.369306,193.50,1.898,0.348238,0.183477,-8.0,74,86,1.59,2.25,123.236,0.000,0.491753,0.000000,0.099917,-1.06,-0.212,-1.698113,0.629981,0.5,-0.129981,0.00,0.000,inf,0.000000,0.0,0.000000
17491,2024-11-24,Argentina Liga Feminina,20:30,Ferro F,Lanus F,1.01,9.35,128.5,1.83,1.83,-18.5,1.96,1.75,SbjYiRWI,0.990099,0.106952,0.546448,0.546448,0.097051,90.674,19.929237,0.219790,1.8,1.643168,0.912871,116.44,1.294,0.229085,0.177036,12.0,187.236,114.480950,0.611426,0.6,1.341641,2.236068,144.10,4.138,3.212354,0.776306,-41.0,71,55,1.64,2.62,93.376,259.368,1.138469,0.000000,0.080050,-1.74,-0.348,-0.028736,0.000000,0.0,0.000000,-2.94,-0.588,-14.200680,0.000000,0.0,0.000000
17492,2024-11-24,Turquia Kbsl Feminina,08:30,Bodrum F,Melikgazi Kayseri F,2.97,1.32,154.5,1.83,1.83,2.5,2.03,1.67,f9GRJtD5,0.336700,0.757576,0.546448,0.546448,0.094276,113.072,11.562447,0.102257,0.6,1.341641,2.236068,106.40,1.588,0.309952,0.195184,-63.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,70,0,1.52,0.00,113.802,0.000,0.543928,0.000000,0.137599,-2.18,-0.436,-4.518349,0.743947,0.6,-0.143947,0.00,0.000,inf,0.000000,0.0,0.000000
17493,2024-11-24,Sérvia 1. Zls Feminino,10:00,Proleter Zrenjanin F,Crvena zvezda F,1.83,1.83,147.5,1.83,1.83,35.5,1.83,1.83,WnoV9BHk,0.546448,0.546448,0.546448,0.546448,0.092896,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,188.048,67.035567,0.356481,1.2,1.643168,1.369306,182.81,2.614,1.129039,0.431920,-3.0,83,101,2.41,1.81,0.000,512.014,0.000000,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,18.29,3.658,0.226900,0.591454,0.7,0.108546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17630,2024-11-24,Europa Eurobasket,13:00,Estônia,Polónia,1.64,2.20,154.5,1.84,1.86,-4.5,2.05,1.60,UaG0kKjk,0.609756,0.454545,0.543478,0.537634,0.064302,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,82,78,2.74,1.43,0.000,0.000,0.206239,0.007644,0.174355,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
17631,2024-11-24,Europa Eurobasket,13:30,Grécia,Reino Unido,1.15,5.11,152.5,1.83,1.91,-13.5,2.05,1.60,ruxIcrq9,0.869565,0.195695,0.546448,0.523560,0.065260,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
17536,19:00,Eua Ncaa,San Diego Toreros,Idaho,1.77,Back Home
17542,16:30,Eua Ncaa,Wisc. Milwaukee,St. Thomas (Minn.),1.69,Back Home
17553,14:00,Eua Ncaa,Portland State,Wofford,2.74,Back Home
17579,12:00,Áustria Liga Zweite,Wagram,Mattersburg,1.56,Back Home
17615,10:30,Europa Wbbl Feminina,LCC F,Liepajas Papirs/LSSS F,1.66,Back Home
17627,22:30,Eua Ncaa,Vanderbilt,Drake,1.64,Back Home
